### [ 동적 모델 클래스 설계 ]
- 1개 데이터셋에 1개의 모델이 아니고 다양한 값을 가질 수 있는 모델 클래스 설계
- 층별 퍼셉트론/뉴런/노드 개수 다양하게
- 층 개수 다양하게

[1] 모듈 로딩 및 데이터 준비 <hr>

In [37]:
# ======================================================================================
# [1-1] 모듈 로딩
# ======================================================================================
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F     # 신경망 함수들(AF, LF, MF) 모듈

from torchinfo import summary
from collections import OrderedDict

# ======================================================================================
# [1-2] 데이터 준비
# ======================================================================================
dataDF = pd.read_csv("../DATA/study_score_multi.csv")
display(dataDF.head())
display(dataDF.info())

,study_hours,sleep_hours,participation,score
0,1.45,5.28,97.4,32.8
1,2.17,5.46,22.4,32.1
2,8.03,5.56,15.6,61.0
3,3.97,4.08,31.2,31.3
4,6.76,7.28,66.8,72.1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   study_hours    5000 non-null   float64
 1   sleep_hours    5000 non-null   float64
 2   participation  5000 non-null   float64
 3   score          5000 non-null   float64
dtypes: float64(4)
memory usage: 156.4 KB


None

[2] 커스텀 모델 클래스 <hr>

In [38]:
# ======================================================================================
# [2-1] 층의 퍼셉트론 수를 동적으로 / HL 2 + OUT 1
# ======================================================================================
#              입력            출력            AF
# 입력층         3               3             -        <= forward()에 전달되는 데이터
# 은닉층         3               ■            ReLU
# 은닉층         ■               ▣           ReLU
# 출력층         ▣               1            -        <= 회귀 모델로 없음
# ======================================================================================
# 클래스이름 : ScoreMode
# 부모클래스 : nn.Module
# 오버라이딩 : __init__(self, hd1, hd2)
#            forward(self, data)
# ======================================================================================
class ScoreModel(nn.Module):
    def __init__(self, hd1, hd2):
        super().__init__()
        self.hd1_layer = nn.Linear(3  , hd1)
        self.hd2_layer = nn.Linear(hd1, hd2)
        self.out_layer = nn.Linear(hd2, 1)

    def forward(self, data):
        out = F.relu(self.hd1_layer(data))
        out = F.relu(self.hd2_layer(out))
        out = self.out_layer(out)

        return out

In [39]:
# 은닉층의 뉴런/퍼셉트론/노드 수에 따른 모델 성능 비교
model1 = ScoreModel(10, 5)
model2 = ScoreModel(50, 70)

In [40]:
# ======================================================================================
# [2-2] 입력, 출력, 은닉층 퍼셉트론 수를 동적으로 / HL 2 + OUT 1
# ======================================================================================
# [2-1] 층의 퍼셉트론 수를 동적으로 / HL 2 + OUT 1
# ======================================================================================
#              입력            출력            AF
# 입력층         △              △             -        <= forward()에 전달되는 데이터
# 은닉층         △               ■            ReLU
# 은닉층         ■               ▣            ReLU
# 출력층         ▣               ○            -        <= 회귀 모델로 없음
# ======================================================================================
# 클래스이름 : CustomModel
# 부모클래스 : nn.Module
# 오버라이딩 : __init__(self, in_in, out_out, hd1_out, hd2_out)
#            forward(self, data)
# ======================================================================================
class CustomModel(nn.Module):
    def __init__(self, in_in, out_out, hd1_out, hd2_out, kind='reg'):
        super().__init__()
        self.hd1_layer = nn.Linear(in_in  , hd1_out)
        self.hd2_layer = nn.Linear(hd1_out, hd2_out)
        self.out_layer = nn.Linear(hd2_out, out_out)
        self.kind = kind

    def forward(self, data):
        out = F.relu(self.hd1_layer(data))
        out = F.relu(self.hd2_layer(out))

        if self.kind in ['reg', 'bin_logis', 'multi_logis']:
            return self.out_layer(out)
            
        elif self.kind == 'bin':  
            # 학습 시 BCELossWithLogits를 쓴다면 sigmoid 빼야 함
            return torch.sigmoid(self.out_layer(out)) 
            
        elif self.kind == 'multi': 
            # 학습 시 CrossEntropyLoss를 쓴다면 softmax 빼야 함
            return F.softmax(self.out_layer(out), dim=1)
            
        else: # 예외 처리
            raise ValueError(f"Unknown kind: {self.kind}")

In [41]:
# 특정 피쳐와 관계 없이 모델
model1 = CustomModel(784, 1, 512, 256, 'multi')   # 입력, 출력, 은닉층1, 은닉층2, 종류

In [ ]:
## =============================================
## [2-3] 입력, 출력, 은닉층 퍼셉트론 수를 동적으로 
## =============================================
## 클래스이름 : DynamicModel
## 부모클래스 : nn.Module
## 매개변수들 : in_out   - 입력층 출력수/피쳐수
##            out_out  - 출력층 출력수
##            *hds     - 은닉층별 출력수
##            kind     - 모델 종류 ('bin', 'multi', 'bin_logits', 'multi_logits')
## =============================================
class DynamicModel(nn.Module):

    ##- 동적 층 수, 뉴런 수 초기화 메서드 
    def __init__(self, in_out, out_out, *hds, kind='logits'):
        super().__init__()
        self.layers = self._make_layers(in_out, out_out, hds)
        self.kind = kind

    ##- 전방향 계산 진행 메서드
    def forward(self, x):
        ##- 은닉층 + ReLU
        for layer in self.layers[:-1]:
            x = F.relu(layer(x))
            
        ##- 출력층 예측값 계산 
        logits = self.layers[-1](x)

        ###- 모델 kind별 출력
        match self.kind:
            case 'bin':
                prob = torch.sigmoid(logits)
                return prob.clamp(1e-7, 1 - 1e-7)
            case 'multi':
                return F.softmax(logits, dim=1)
            case 'logits' | 'bin_logis' | 'multi_logis':
                return logits
            case _:
                raise ValueError(f"Unknown kind: {self.kind}")

    ##- 동적 층 생성 인스턴스 메서드 
    def _make_layers(self, in_out, out_out, hd):
        dims = [in_out, *hd, out_out]
        return nn.ModuleList([
            nn.Linear(dims[i], dims[i+1]) for i in range(len(dims) - 1)
        ])

In [43]:
m1 = DynamicModel(1,3,5,7)
print(m1)

DynamicModel(
  (layers): ModuleList(
    (0): Linear(in_features=1, out_features=5, bias=True)
    (1): Linear(in_features=5, out_features=7, bias=True)
    (2): Linear(in_features=7, out_features=3, bias=True)
  )
)
